#Movie Recommandation System 




In [ ]:
#Import Depends 
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv('./movies.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [ ]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
data.shape

(4803, 24)

In [ ]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
#Features Selection
chosen_features = [ 'genres', 'keywords', 'tagline','cast', 'director' , 'overview']

In [ ]:
data[chosen_features].isnull().sum()

genres       28
keywords    412
tagline     844
cast         43
director     30
overview      3
dtype: int64

In [ ]:
#Trait Missing Values 
for col in chosen_features : 
  data[col] = data[col].fillna('')

In [ ]:
data[chosen_features].isnull().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
overview    0
dtype: int64

In [ ]:
features = data[chosen_features]

In [ ]:
features.head()

,genres,keywords,tagline,cast,director,overview
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,"In the 22nd century, a paraplegic Marine is di..."
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,"Captain Barbossa, long believed to be dead, ha..."
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,A cryptic message from Bond’s past sends him o...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,Following the death of District Attorney Harve...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,"John Carter is a war-weary, former military ca..."


In [ ]:
#Combine Features
combined_features = features['genres']
for feature in chosen_features : 
  if feature == 'genres' : continue 
  combined_features = combined_features + ' ' + features[feature]

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
#Coverting Text -> Feature Vector
vectorizer = TfidfVectorizer() 

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
#Cosine Similarity #Similarity Confidence score 
similarities  = cosine_similarity(feature_vectors)

In [ ]:
similarities.shape

(4803, 4803)

In [ ]:
#Give an Input 
movie_name = input('Enter Your Fav Movie Title : ')

KeyboardInterrupt: ignored

In [ ]:
#Rendering a the recommended movies list  
all_movies_titles  = data['title'].tolist()
print(all_movies_titles)

In [ ]:
#Finding the closest match to the input 
matches = difflib.get_close_matches(movie_name,all_movies_titles)
print("Select (Number) one of the following movies ")
for (i,match) in enumerate(matches) : 
  print(i , "." , match)
k = int(input())
movie_name = matches[k]
print("The movie : ",movie_name)

In [ ]:
#Searching for the index of the movie in the datset 
index = data[data['title'] == movie_name ]['index'].values[0]
print(index)

In [ ]:
#Rendering a the recommended movies list  
similarity_score = list(enumerate(similarities[index]))
print(similarity_score)

In [ ]:
list_sorted  = sorted(similarity_score,key = lambda x : x[1] , reverse = True )
print(list_sorted)

In [ ]:
#The Similar Movies 
print('For You Movies : FYM')
i = 1
for movie in list_sorted : 
  index  = movie[0]
  movie_title  = data[data.index == index]['title'].values[0]
  print(i,'.',movie_title)
  i= i+1
  if (i>11) : break

In [ ]:
#Recap 
movie_name = input('Enter Your Fav Movie Title : ')

matches = difflib.get_close_matches(movie_name,all_movies_titles)
print("Select (Number) one of the following movies ")
for (i,match) in enumerate(matches) : 
  print(i , "." , match)
k = int(input())
movie_name = matches[k]
print("The movie : ",movie_name)

index = data[data['title'] == movie_name ]['index'].values[0]
similarity_score = list(enumerate(similarities[index]))
list_sorted  = sorted(similarity_score,key = lambda x : x[1] , reverse = True )
list_sorted.pop(0)
print('---------------------------------------------------------------')
print('For You Movies : FYM')
i = 1
for movie in list_sorted : 
  index  = movie[0]
  movie_title  = data[data.index == index]['title'].values[0]
  print(i,'.',movie_title)
  i= i+1
  if (i>10) : break

Enter Your Fav Movie Title : Drive
Select (Number) one of the following movies 
0 . Drive
1 . Driven
2 . Sex Drive
0
The movie :  Drive
---------------------------------------------------------------
For You Movies : FYM
1 . Only God Forgives
2 . A Most Violent Year
3 . Bronson
4 . The Neon Demon
5 . Tadpole
6 . Inside Llewyn Davis
7 . Aqua Teen Hunger Force Colon Movie Film for Theaters
8 . The Muse
9 . The Place Beyond the Pines
10 . Triple 9
